In [48]:
from sklearn import datasets
import math
import numpy as np
from scipy import stats
import time

# Agglomerative Clustering
------------------------------------------
*Agglomerative clustering* merupakan suatu teknik dalam melakukan *clustering* yang menggunakan pendekatan *hierarchical*. Ide utama dari teknik ini adalah dengan melakukan penggabungan dua buah *cluster* dengan jarak terdekat pada setiap iterasi hingga diperoleh banyak *cluster* sesuai yang dikehendaki.

Dalam notasi *pseudocode*, algoritma *agglomerative clustering* adalah sebagai berikut.

```
WHILE (length(clusters_now) > nb_target_cluster) DO
    pair_merged = get_shortest_distance_pair(clusters_now)
    merge_cluster(pair_merged)
    update_cluster(clusters_now)
```

Terdapat beberapa pendekatan dalam menentukan pasangan *cluster* dengan jarak terdekat. Pendekatan tersebut adalah sebagai berikut.
1. ***Complete linkage***. Dilakukan dengan menghitung jarak antara dua titik terjauh pada kedua *cluster*.
![complete linkage](complete.png "Complete linkage")
2. ***Single linkage***. Dilakukan dengan menghitung jarak antara dua titik terdekat pada kedua *cluster*.
![single linkage](single.png "Single linkage")
3. ***Average linkage***. Dilakukan dengan menghitung jarak rata-rata antara semua pasangan titik pada kedua cluster.
![average linkage](average.png "Average linkage")
4. ***Average-group linkage***. Dilakukan dengan menghitung jarak antara *centroid* dari kedua *cluster*.
![average group linkage](average_group.png "Average-group linkage")

Teknik penghitungan jarak juga beragam. Dalam implementasi ini, diterapkan jarak *manhattan*, *euclidean*, dan *cosine*.

# Class

In [50]:
import numpy as np
import math

class AgglomerativeClustering:
    '''
    Kelas untuk mengakomodasi nilai metode agglomerative clustering
    '''
    
    # Nilai default parameter
    n_clusters = 2
    linkage = 'complete'
    metrics = 'euclidean'
    
    available_metrics = ['euclidean', 'manhattan', 'cosine']
    available_linkage = ['complete', 'single', 'average_group', 'average']
    
    def __init__(self, n_clusters=n_clusters, linkage=linkage, metrics=metrics):
        '''
        Inisiasi kelas. Parameter yang dibutuhkan untuk setiap kelas diinisiasi atau diisi dengan nilai default 
        '''
        if n_clusters <= 0:
            raise Exception('n_clusters must be higher than 0')
        if metrics not in self.available_metrics:
            raise Exception('No metrics \'' + str(metrics) + '\'. Available metrics '+ str(self.available_metrics))
        if linkage not in self.available_linkage:
            raise Exception('No linkage \'' + str(linkage) + '\'. Available linkage '+ str(self.available_linkage))
        self.metrics = metrics
        self.n_clusters = n_clusters
        self.linkage = linkage
        
    def __euclidean_distance(self, data1, data2):
        '''
        Fungsi untuk menghitung euclidean distance di antara dua vector dengan panjang yang sama
        '''
        sum = 0
        if (len(data1) == len(data2)):
            for x1, x2 in zip(data1, data2):
                sum += (x1 - x2)**2
            dist = math.sqrt(sum)
            return dist
        else:
            raise Exception('Length doesn\'t match')

    def __manhattan_distance(self, data1, data2):
        '''
        Fungsi untuk menghitung manhattan distance di antara dua vector dengan panjang yang sama
        '''
        sum = 0
        if (len(data1) == len(data2)):
            for x1, x2 in zip(data1, data2):
                sum += abs(x1 - x2)
            return sum
        else:
            raise Exception('Length doesn\'t match')
            
    def __cosine_distance(self, data1, data2):
        '''
        Fungsi untuk menghitung cosine distance di antara dua vector dengan panjang yang sama
        '''
        return np.dot(data1, data2) / (np.linalg.norm(data1) * np.linalg.norm(data2))

    def __get_distance(self, data1, data2, metrics):
        '''
        Fungsi untuk menghitung jarak dua vector dengan metric pengukuran jarak yang telah ditentukan
        '''
        if (metrics == 'euclidean'):
            dist = self.__euclidean_distance(data1, data2)
        elif (metrics == 'manhattan'):
            dist = self.__manhattan_distance(data1, data2)
        elif (metrics == 'cosine'):
            dist = self.__cosine_distance(data1, data2)
        else:
            raise Exception('Metrics not defined')
        return dist
    
    def __complete_linkage(self, cluster1, cluster2, dist_matrix):
        '''
        Fungsi untuk menghitung jarak antara dua cluster dengan pendekatan complete linkage
        '''
        max_dist = 0
        for v1 in cluster1:
            for v2 in cluster2:
                if (max_dist < dist_matrix[v1][v2]):
                    max_dist = dist_matrix[v1][v2]
        return max_dist

    def __single_linkage(self, cluster1, cluster2, dist_matrix):
        '''
        Fungsi untuk menghitung jarak antara dua cluster dengan pendekatan single linkage
        '''
        min_dist = None
        for v1 in cluster1:
            for v2 in cluster2:
                if (min_dist is None) or (min_dist > dist_matrix[v1][v2]):
                    min_dist = dist_matrix[v1][v2]
        return min_dist

    def __average_linkage(self, cluster1, cluster2, dist_matrix):
        '''
        Fungsi untuk menghitung jarak antara dua cluster dengan pendekatan average linkage
        '''
        sum_dist = 0
        count_dist = 0
        for v1 in cluster1:
            for v2 in cluster2:
                sum_dist += dist_matrix[v1][v2]
                count_dist += 1
        return float(sum_dist)/float(count_dist)

    def __group_average_linkage(self, cluster1, cluster2, data, distance):
        '''
        Fungsi untuk menghitung jarak antara dua cluster dengan pendekatan average group linkage
        '''
        data1 = [data[i] for i in cluster1]
        data2 = [data[i] for i in cluster2]

        avg1 = np.mean(data1, axis = 0)
        avg2 = np.mean(data2, axis = 0)

        return self.__get_distance(avg1, avg2, distance)
    
    def __calculate_distance_matrix(self, data, metrics):
        '''
        Fungsi untuk menghitung distance matrix untuk semua pasangan vector di dalam data
        '''
        dist_matrix = []
        for idx1, data1 in enumerate(data):
            curr_dist_matrix = []
            for idx2, data2 in enumerate(data):
                if (idx1 > idx2):
                    curr_dist_matrix.append(dist_matrix[idx2][idx1])
                else:
                    dist = self.__get_distance(data1, data2, metrics)
                    curr_dist_matrix.append(dist)
            dist_matrix.append(curr_dist_matrix)
        return dist_matrix
        
    def fit_predict(self, data):
        '''
        Fungsi untuk melakukan clustering secara agglomerative
        '''
        
        # preprocessing distance matrix
        if (self.linkage != 'average_group'):
            dist_matrix = self.__calculate_distance_matrix(data, self.metrics)
        # inisiasi cluster dengan satu elemen per cluster awal 
        clusters = [[i] for i, c in enumerate(data)]

        # melakukan iterasi hingga diperoleh jumlah cluster sesuai yang dikehendaki
        while(len(clusters) > self.n_clusters):
            min_dist = None
            merge_pair = (0, 0)
            
            # mencari cluster dengan jarak terdekat untuk di-merge
            for idx1, c1 in enumerate(clusters):
                for idx2, c2 in enumerate(clusters[(idx1 + 1) :]):
                    if (self.linkage == 'single'):
                        dist = self.__single_linkage(c1, c2, dist_matrix)
                    elif (self.linkage == 'complete'):
                        dist = self.__complete_linkage(c1, c2, dist_matrix)
                    elif (self.linkage == 'average'):
                        dist = self.__average_linkage(c1, c2, dist_matrix)
                    elif (self.linkage == 'average_group'):
                        dist = self.__group_average_linkage(c1, c2, data, self.metrics)
                    else:
                        raise Exception('Linkage not defined')
                    if (min_dist == None) or (dist < min_dist):
                        min_dist = dist
                        merge_pair = (idx1, idx1 + 1 + idx2)
            
            # merge pasangan cluster dengan jarak terdekat
            result_cluster = []
            for idx, c in enumerate(clusters):
                if idx not in merge_pair:
                    result_cluster.append(c)

            result_cluster.append(clusters[merge_pair[0]] + clusters[merge_pair[1]])

            clusters = result_cluster

        # menampilkan hasil clustering
        result_per_item = np.full(len(data), 0)
        for idx, clust in enumerate(clusters):
            result_per_item[clust] = idx

        return result_per_item
    
    '''
    Getter untuk parameter
    '''
    def get_n_cluster(self):
        return self.n_clusters
    
    def get_linkage(self):
        return self.linkage
    
    def get_metrics(self):
        return self.metrics

## Experiments
Berikut adalah eksperimen yang dilakukan untuk melakukan *clustering* terhadap *dataset* iris. Digunakan *euclidean distance* dan berbagai variasi perhitungan *linkage* antara dua cluster untuk menerapkan metode *agglomerative clustering*.

In [36]:
def purity(cluster_pred, label):
    data_per_cluster = [[] for i in range(len(set(cluster_pred)))]
    for i,x in enumerate(cluster_pred):
        data_per_cluster[x].append(label[i])

    sum = 0
    for clust in data_per_cluster:
        sum += stats.mode(clust)[1][0]

    return sum/len(cluster_pred)

In [11]:
iris = datasets.load_iris()
data = iris.data
label = iris.target

#### Clustering dengan Complete Linkage
Pada percobaan ini, digunakan pendekatan *complete linkage* untuk menghitung jarak antar pasangan cluster. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [52]:
aglo = AgglomerativeClustering(n_clusters=3, linkage='complete', metrics='euclidean')
start = time.time()
pred = aglo.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.7402849197387695 s ----
Cluster prediction:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 0 2 0 2 0 2 0 0 0 0 2 0 2 0 0 2 0 2 0 2 2
 2 2 2 2 2 0 0 0 0 2 0 2 2 2 0 0 0 2 0 0 0 0 0 2 0 0 2 2 2 2 2 2 0 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Purity: 0.84


#### Clustering dengan Single Linkage
Pada percobaan ini, digunakan pendekatan *single linkage* untuk menghitung jarak antar pasangan cluster. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [53]:
aglo = AgglomerativeClustering(n_clusters=3, linkage='single', metrics='euclidean')
start = time.time()
pred = aglo.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.6014959812164307 s ----
Cluster prediction:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Purity: 0.68


#### Clustering dengan Average Linkage
Pada percobaan ini, digunakan pendekatan *complete linkage* untuk menghitung jarak antar pasangan cluster. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [55]:
aglo = AgglomerativeClustering(n_clusters=3, linkage='average', metrics='euclidean')
start = time.time()
pred = aglo.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 1.103651523590088 s ----
Cluster prediction:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 1 1 1 1 2 1 1 1 1
 1 1 2 2 1 1 1 1 2 1 2 1 2 1 1 2 2 1 1 1 1 1 2 1 1 1 1 2 1 1 1 2 1 1 1 2 1
 1 2]
Purity: 0.9066666666666666


#### Clustering dengan Average-group Linkage
Pada percobaan ini, digunakan pendekatan *average-group linkage* untuk menghitung jarak antar pasangan cluster. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [56]:
aglo = AgglomerativeClustering(n_clusters=3, linkage='average_group', metrics='euclidean')
start = time.time()
pred = aglo.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 43.371429681777954 s ----
Cluster prediction:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 0 0 0 2 0 0 0 0
 0 0 2 2 0 0 0 0 2 0 2 0 2 0 0 2 2 0 0 0 0 0 2 0 0 0 0 2 0 0 0 2 0 0 0 2 0
 0 2]
Purity: 0.9066666666666666


## Hasil dan Analisis
Dari keempat eksperimen di atas, dapat ditarik kesimpulan bahwa untuk dataset iris, metode *agglomerative* dapat diterapkan untuk melakukan *clustering*. Penggunaan teknik *average linkage* dan *average-group linkage* untuk menghitung jarak antara dua cluster menghasilkan *cluster* dengan *purity* tertinggi, yaitu 0.9067. Namun, *average-group linkage* membutuhkan waktu eksekusi yang lebih lama dibandingkan *average linkage*. Hal ini dikarenakan teknik *average-group linkage* tidak dapat menggunakan hasil *preprocessing distance matrix* sehingga perlu dilakukan komputasi ulang dalam menghitung jarak antara dua *cluster* .